# Run Spark Locally


In [2]:
#Andy Ham aham
#Problem 4A
from pyspark.sql import SparkSession
import re

spark  = SparkSession.builder.master('local').appName('SOU').getOrCreate()
df = spark.read.json('/project/cmsc25025/sou/speeches.json')

print df.rdd.map(lambda x: re.split('(?<=[.!?]) +', x['text'])).collect()
#print df.rdd.map(lambda x: len(x['text'].split('. '))).collect()

spark.stop()

SyntaxError: invalid syntax (<ipython-input-2-6a4d8a25bf1f>, line 9)

In [3]:
#Problem 4B
%matplotlib inline

from pyspark.sql import SparkSession
import numpy as np
import matplotlib.pyplot as plt
import re

spark  = SparkSession.builder.master('local').appName('SOU').getOrCreate()
df = spark.read.json('/project/cmsc25025/sou/speeches.json')

num_sentences = df.rdd.map(lambda x: len(re.split('(?<=[.!?]) +', x['text']))).collect()
#num_sentences = df.rdd.map(lambda x: len(x['text'].split('. '))).collect()

mean_sentence_len = df.rdd.map(lambda x: len(x['text'].split(' '))/len(re.split('(?<=[.!?]) +', x['text']))).collect()
#mean_sentence_len= df.rdd.map(lambda x: len(x['text'].split(' '))/len(x['text'].split('. '))).collect()

yearsX = df.rdd.map(lambda x: int(x['year'])).collect()

#print num_sentences
#print mean_sentence_len
#print yearsX

combined = df.rdd.map(lambda x: (int(x['year']), len(re.split('(?<=[.!?]) +', x['text'])), len(x['text'].split(' '))/len(re.split('(?<=[.!?]) +', x['text'])))).collect()

print "Format is: (Year, # of sentences, mean sentence length in words) \n"
print sorted(combined)

b1_hat_num = np.cov(yearsX, num_sentences)[0][1] / np.cov(yearsX, num_sentences)[0][0]
b0_hat_num = (sum(num_sentences)/len(num_sentences)) - (b1_hat_num * (sum(yearsX)/len(yearsX)))

b1_hat_mean = np.cov(yearsX, mean_sentence_len)[0][1] / np.cov(yearsX, mean_sentence_len)[0][0]
b0_hat_mean = (sum(mean_sentence_len)/len(mean_sentence_len)) - (b1_hat_mean * (sum(yearsX)/len(yearsX)))

y_val1790_num = (b1_hat_num*1790) + b0_hat_num
y_val2013_num = (b1_hat_num*2013) + b0_hat_num

y_val1790_mean = (b1_hat_mean*1790) + b0_hat_mean
y_val2013_mean = (b1_hat_mean*2013) + b0_hat_mean

plt.figure(1)
plt.plot(yearsX, num_sentences, '.')
plt.plot([1790, 2013], [y_val1790_num, y_val2013_num]) #regression line
plt.title('Number of sentences by year')
plt.xlabel('Year')
plt.ylabel('# of sentences')

plt.figure(2)
plt.plot(yearsX, mean_sentence_len, '.')
plt.plot([1790, 2013], [y_val1790_mean, y_val2013_mean]) #regression line
plt.title('Average sentence length by year')
plt.xlabel('Year')
plt.ylabel('Average sentence length (# of words)')

print "\n\nThe # of sentences in a SOU address has an upward trend with time, whereas the mean sentence length of "
print "of sentences goes down. It makes sense that there would be generally be an inverse proportion between the "
print "number of sentences and the average sentence length if the addresses were to have the same amount of content"
print "Also noted in 4c, from 1801 to 1912 SOU addresses were written, which makes it easier for presidents to have"
print "less number of sentences/longer average sentence length thanks to semi colons and flowing prose. It's hard to"
print "do that with oral speeches, and succint phrases are easier for people to pay attention and understand."

spark.stop()

SyntaxError: invalid syntax (<ipython-input-3-a7a0d117621b>, line 26)

In [4]:
#Problem 4C compute 2 regressions of TOTAL NUM WORDS in SOU vs 1790-1912, and 1913-2013
%matplotlib inline
from pyspark.sql import SparkSession
import numpy as np
import matplotlib.pyplot as plt
import re

spark  = SparkSession.builder.master('local').appName('SOU').getOrCreate()
df = spark.read.json('/project/cmsc25025/sou/speeches.json')

num_words= df.rdd.map(lambda x: (int(x['year']), len(x['text'].split(' ')))).collect()

yearsX = sorted(df.rdd.map(lambda x: int(x['year'])).collect())

num_words.sort(key=lambda x: x[0])
# tuples of (year, wordcount)

before = num_words[:123]
after = num_words[123:]

before_year, before_words = zip(*before)
after_year, after_words= zip(*after)

b1_hat_before = np.cov(before_year, before_words)[0][1] / np.cov(before_year, before_words)[0][0]
b0_hat_before = (sum(before_words)/len(before_words)) - (b1_hat_before * (sum(before_year)/len(before_year)))

b1_hat_after = np.cov(after_year, after_words)[0][1] / np.cov(after_year, after_words)[0][0]
b0_hat_after = (sum(after_words)/len(after_words)) - (b1_hat_after * (sum(after_year)/len(after_year)))

y_val1790_before= (b1_hat_before*1790) + b0_hat_before
y_val1912_before = (b1_hat_before*1912) + b0_hat_before

y_val1913_after = (b1_hat_after*1913) + b0_hat_after
y_val2013_after = (b1_hat_after*2013) + b0_hat_after

plt.figure(1)
plt.plot(before_year, before_words, '.')
plt.plot([1790, 1912], [y_val1790_before, y_val1912_before]) #regression line
plt.title('# of words in SOU by year 1790-1912')
plt.xlabel('Year')
plt.ylabel('# of words')

plt.figure(2)
plt.plot(after_year, after_words, '.')
plt.plot([1913, 2013], [y_val1913_after, y_val2013_after]) #regression line
plt.title('# of words in SOU by year 1913-2013')
plt.xlabel('Year')
plt.ylabel('# of words')

print "SOU addresses from 1790 to 1912 experience a clearly upward trend in the number of words, from a" 
print "couple thousand to over 25,000 words. However, from 1913 to 2013, the # of words is relatively constant"
print "around 5 or 6k. This disparity arises from when Thomas Jefferson in 1801 thought giving a speech in person"
print "to be too monarchial and instead wrote the address and sent it to Congress to be read by a clerk."
print "In 1913, Woodrow Wilson re-established the practice of giving the addresses in person. It seems easier"
print "to add more content while writing an address, since the president is somewhat detached from the actual"
print "delivery of the speech, but giving an address the president needs to be aware of how long his speech is"
print "Wilson's first SOU address was probably the model/template for all future addresses, and the following"
print "presidents tried to keep a similar time/word limit. The exceptions are in 1981 when Jimmy Carter wrote in"
print "the longest SOU address ever, and another outlier by Harry Truman when he wrote in his address in 1946."

spark.stop()

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-4-3cdf2f670d84>, line 50)

In [5]:
#PROBLEM 4D
#Which President has the longest sentences on average? Which has the shortest sentences?
#Compute the median, 25% and 75% quantiles across all Presidents. What was
#the longest and shortest sentence ever spoken (or written) in a SOU?
#numwords/numsentences
%matplotlib inline
from pyspark.sql import SparkSession
from operator import add
import re
import numpy as np

spark  = SparkSession.builder.master('local').appName('SOU').getOrCreate()
df = spark.read.json('/project/cmsc25025/sou/speeches.json')


speech = df.rdd.map(lambda x: (x['president'], int(x['year']), len(x['text'].split(' ')),re.split('(?<=[.!?]) +', x['text']))).collect()
# (u'James Monroe', 1821, 5765words, 169sentenc), ...

speechavglen = df.rdd.map(lambda x: (x['president'], len(x['text'].split(' '))/len(re.split('(?<=[.!?]) +', x['text'])))).reduceByKey(add).collect()
speechnumyears = df.rdd.map(lambda x: (x['president'], 1)).reduceByKey(add).collect()


final = spark.sparkContext.parallelize(speechavglen + speechnumyears).reduceByKey(lambda a,b: a/b).collect()
#list of (president, avg wordcount per sentence) tuples

sortedfinal = sorted(final, key=lambda x: x[1])

#print sortedfinal
print "President with the longest sentences on average is: "
print sortedfinal[len(sortedfinal)-1]
print "\nPresident with the shortest sentences on average is: "
print sortedfinal[0]
print "\nPresident with the median is: "
print sortedfinal[len(sortedfinal)/2]
print "\nPresident at the 25th percentile is: "
print sortedfinal[len(sortedfinal)*1/4]
print "\nPresident at the 75th percentile is: "
print sortedfinal[len(sortedfinal)*3/4]

def longest_sentence(sentences):
    maxlength = 0
    for sentence in sentences:
        if len(sentence.split(' ')) > maxlength:
            maxlength = len(sentence.split(' '))
            maxs = (sentence, maxlength)
    return maxs

def shortest_sentence(sentences):
    minlength = 1000
    for sentence in sentences:
        if len(sentence.split(' ')) < minlength:
            minlength =len(sentence.split(' '))
            mins = (sentence, minlength)
    return mins

shortest = df.rdd.map(lambda x: shortest_sentence(re.split('(?<=[.!?]) +', x['text']))).collect()
longest = df.rdd.map(lambda x: longest_sentence(re.split('(?<=[.!?]) +', x['text']))).collect()


#need to account for " mark
longest2 = sorted(longest)[len(longest)-1][0].split('\"')[1]

print "\nLongest sentence in SOU: "
print (longest2, len(longest2.split(" ")))
print "\nShortest sentence in SOU (considering 2 word count to be minimum, there are a few): "
#print sorted(shortest, key=lambda x: x[1])[156]
print sorted(shortest, key=lambda x: x[1])[149]
                                
spark.stop()

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-5-21d0c8f29a80>, line 29)